In [1]:
# Import the required libraries
import joblib
import datetime
import config
import input
import preprocessing_functions as pf
import pandas as pd
from AQI_form import AQIForm
from flask import Flask
from flask import request
from flask import jsonify
from flask import session
from flask import url_for
from flask import redirect
from flask import render_template
from AQI_form import AQIForm

# Read the model
#AQIModel = joblib.load("Decision_Tree_AQI-Project.pkl")
AQIModel = "bla"

# Prediction function
def make_prediction(model, sample_json):
    # parse input from request
    PredictionDate = sample_json['PredictionDate']
    PredictionCity = sample_json['PredictionCity']

    PredictionDate = pf.form_format_date(PredictionDate)

    input.model_weather_data(PredictionDate, PredictionCity)

    # CSV-File ins Dataframe einlesen
    df_weather = pd.read_csv(config.csv_temp_weather)
    
    ##Code zum extrahieren der Daten für das Model => precipprob, windgust, visibility
    precipprob = df_weather['precipprob']
    windgust = df_weather['windgust']
    visibility = df_weather['visibility']    
    
    # Make an input vector
    weather = [[precipprob, windgust, visibility]]

    # Predict
    #PM10_prediction = model.predict(weather)

    #return PM10_prediction
    return weather

app = Flask(__name__)
app.config['SECRET_KEY'] = 'mysecretkey'

@app.route("/", methods=['GET','POST'])
def index():
    form = AQIForm()

    if form.validate_on_submit():
        session['PredictionDate'] = form.PredictionDate.data
        session['PredictionCity'] = form.PredictionCity.data

        return redirect(url_for("prediction"))
        
    return render_template("home.html", form=form)

@app.route('/prediction')
def prediction():
    content = {'PredictionDate': session['PredictionDate'],'PredictionCity': str(session['PredictionCity'])}

    results = make_prediction(AQIModel, content)

    return render_template('prediction.html', results=results)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.1.123:8080
Press CTRL+C to quit
127.0.0.1 - - [21/May/2023 12:04:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2023 12:04:38] "POST / HTTP/1.1" 302 -


 - Running query URL:  https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/47.39,8.54/2023-05-22/2023-05-22?&unitGroup=metric&contentType=csv&include=days&key=XB2AJ7B74CP7PS8RWU3HY8WME



127.0.0.1 - - [21/May/2023 12:04:38] "GET /prediction HTTP/1.1" 200 -
